In [0]:
%pip install openai unidecode gspread==5.12.4 tiktoken mlflow
%restart_python
%load_ext autoreload
%autoreload 2 

In [0]:
%restart_python
%load_ext autoreload
%autoreload 2 

In [0]:
from abc import ABC, abstractmethod
import time
import mlflow
from contextlib import contextmanager
import json
import pandas as pd
import datetime
import re
import openai
from openai import OpenAI
import gspread
import random
import logging
import os
from unidecode import unidecode
from pathlib import Path
import pyspark
from pyspark.sql.functions import *
from pyspark.sql.types import *
from functools import reduce
from typing import *
import tiktoken
import io


from general_config import *
from ml_tracker import MLTracker
from aso_config import *
from aso_localizer import *
import numpy as np
#from in_game_localizer import InGameLocalizer
#from InGame_Config import * 

In [0]:
%run "./authenticationScript"

In [0]:
#TODO update authenticatinon script and secrets!!
gsheet_client = get_gspread_client_from_secret_old()
gpt_client = get_model_client() 

In [0]:
## TEST

october_urls = [{'Game':'Cookie Jam', "URL": "https://docs.google.com/spreadsheets/d/1rhG8J_qsGzntrUu4BoI4kOCqrUY80xmm_jl7b7SPcss/edit?gid=980819314#gid=980819314"}]

In [0]:
base_request = {"RowFingerprint":"aso_october_cj",
                    "Timestamp":datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                    "SubmitterEmail":"krista@jamcity.com",
                    "DueDate":"9/4/25",
                    'LocType':"ASO", 
                    'TargetLanguages':"",
                    "QAFlag":""}

cfg = {
    "input": 
        {
            "required_tabs": ["ios","android"],
            "ios_header_rows": 3, 
            "android_header_rows": 3
        },
    ##add more formatting data for header rows
    "char_limit_policy": "strict",
    "output_sheets":
        ["formatted ios", "formatted android", "long results",'wide results'],
    ##add more formatting info for output sheets
    "log_full_artifacts":True,
    "qc": {"enabled": True, "max_retries": 15,},
}
localizers= []
for i in october_urls:
    i = {**base_request, **i}
    localizer = ASOLocalizer(request = i, gsheet_client =gsheet_client, gpt_client = gpt_client, cfg=cfg)
    localizers.append(localizer)

In [0]:
localizer = localizers[0] # for testing this

In [0]:
## QC loop not quite working

In [0]:
output = localizer.run()

In [0]:
#maybe add a note about QC flag not completing it all

output = {'status': 'SUCCEEDED', 'run_id': '1f5484e426e749f38b4c111d1f1daef5'}

In [0]:
# ok lastly, write to the table
#output = {'status': 'SUCCEEDED', 'run_id': '1f5484e426e749f38b4c111d1f1daef5'}
#localizer.request ... input


In [0]:
#localizer.

In [0]:
#formatted = (localizer.unioned_long)

In [0]:
report = localizer.qc_checks(localizer.unioned_long)

In [0]:
issues = report.get("issues")


by_lang: Dict[str, List[Dict[str, Any]]] = {}
for it in issues:
    lang = it.get("language") or it.get("language_cd") or "unknown"
    by_lang.setdefault(lang, []).append(it)


In [0]:
by_lang

In [0]:

new_rows: List[Dict[str, Any]] = []
for lang, items in by_lang.items():
    # Build a tiny JSON input of just the broken rows
    # Using the *same output schema* you expect: [{row_idx, lang_cd: "new text"}]
    payload = []
    for it in items:
        payload.append({
            "row_idx": it["row_idx"],
            "target_char_limit": it["target_char_limit"],
            "en_US": it.get("en_text", ""),  # if you stored English
        })
    slug = json.dumps(payload, ensure_ascii=False)
    strict_prompt = localizer._generate_qc_prompt(lang,slug)
    out_str, usage = localizer._call_model_batch(strict_prompt)
    fixed_list =localizer._parse_model_json_block(out_str)

    lang_cd = localizer.lang_map[lang]
    fix_df = pd.DataFrame(fixed_list)
    if "row_idx" in fix_df.columns and lang_cd in fix_df.columns:
        fix_df = fix_df[["row_idx", lang_cd]].rename(columns={lang_cd: "translation"})
        fix_df["language"] = lang
        fix_df["language_cd"] = lang_cd
        new_rows.append(fix_df)

if new_rows:
    fixes = pd.concat(new_rows, axis=0, ignore_index=True)


In [0]:
formatted

In [0]:
fixes

In [0]:
fixes = fixes.rename(columns={"translation": "translation_fix",})

In [0]:
fixes

In [0]:
key_cols = ["row_idx", "language_cd",'language']
updated = (
    formatted.drop(columns=["translation"], errors="ignore")
    .merge(fixes, on=key_cols, how="left", suffixes=("", "_fix"))
)
# prefer the fix where present
#updated["translation"] = updated["translation"].where(updated["translation_fix"].isna(),
                                                        #updated["translation_fix"])
#updated = updated.drop(columns=["translation_fix"])

In [0]:
import pandas as pd
import numpy as np

def _normalize_lang_col(df: pd.DataFrame) -> pd.DataFrame:
    # tolerate either language_cd or target_lang_cd
    if "language_cd" in df.columns:
        return df
    if "target_lang_cd" in df.columns:
        return df.rename(columns={"target_lang_cd": "language_cd"})
    return df

# formatted: long DF with ['row_idx','language_cd','translation',...]
# fixes: DF with new translations for subset rows; must end up as
#        ['row_idx','language_cd','translation_fix']
def apply_translation_fixes(formatted: pd.DataFrame, fixes: pd.DataFrame) -> pd.DataFrame:
    formatted = _normalize_lang_col(formatted.copy())
    fixes = fixes.copy()

    # Ensure fixes has language_cd and a translation_fix column
    if "language_cd" not in fixes.columns and "target_lang_cd" in fixes.columns:
        fixes = fixes.rename(columns={"target_lang_cd": "language_cd"})

    # If fixes still lacks language_cd but has a single language, you can inject it:
    # if "language_cd" not in fixes.columns and "language" in fixes.columns:
    #     fixes = fixes.rename(columns={"language": "language_cd"})

    # Normalize fix col name
    if "translation_fix" not in fixes.columns:
        # typical case: fixes has 'translation' (from parsed model output)
        if "translation" in fixes.columns:
            fixes = fixes.rename(columns={"translation": "translation_fix"})
        elif "translation_fixed" in fixes.columns:   # earlier variant
            fixes = fixes.rename(columns={"translation_fixed": "translation_fix"})
        else:
            # nothing to apply
            return formatted

    key_cols = ["row_idx", "language_cd"]

    # Sanity: ensure keys exist
    for k in key_cols:
        if k not in formatted.columns or k not in fixes.columns:
            # Nothing to merge if keys missing
            return formatted

    # DO NOT drop 'translation' before merge; merge fixes to the right
    updated = formatted.merge(fixes[key_cols + ["translation_fix"]],
                              on=key_cols, how="left")

    # Prefer fixed where present
    if "translation_fix" in updated.columns:
        updated["translation"] = np.where(
            updated["translation_fix"].notna(),
            updated["translation_fix"],
            updated["translation"]
        )
        updated = updated.drop(columns=["translation_fix"])

    return updated

In [0]:
test_updated = apply_translation_fixes(formatted,fixes)

In [0]:
test_updated

In [0]:
updated_translation_fi

In [0]:
updated['translation_fix'] = updated['translation_fix'].where(updated['translation_fix'].notna())

In [0]:
fixes

In [0]:
slug

In [0]:
#strict_prompt = localizer._generate_qc_prompt('Portuguese (Brazil)',slug)

In [0]:
strict_prompt

out_str, usage = localizer._call_model_batch(strict_prompt)

In [0]:
fixed_list = localizer._parse_model_json_block(out_str)
lang = 'Portuguese (Brazil)'

lang_cd = localizer.lang_map[lang]
fix_df = pd.DataFrame(fixed_list)
if "row_idx" in fix_df.columns and lang_cd in fix_df.columns:
    fix_df = fix_df[["row_idx", lang_cd]].rename(columns={lang_cd: "translation"})
    fix_df["language"] = lang
    fix_df["language_cd"] = lang_cd
    new_rows.append(fix_df)


In [0]:
fix_df

In [0]:
#mlflow.end_run("3739efa5a69945e09c0e817f98137087")
run = mlflow.active_run()
mlflow.end_run()

In [0]:
print(mlflow.active_run())
mlflow.end_run()

In [0]:
results

In [0]:
results = []
for loc in localizers:
    print(f"Running for {loc.request['Game']}")
    results.append(loc.run())
    #postprocessed = loc.postprocess(results)
    #print(loc.write_outputs(postprocessed))

In [0]:
results

In [0]:
"""
request =  {"RowFingerprint":"testing_aso_run_v2",
                    "Timestamp":datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                    "SubmitterEmail":"krista@jamcity.com",
                    "DueDate":"8/29/25",
                    'LocType':"ASO", 
                    'Game':"Panda Pop", 
                    'TargetLanguages':"",
                    "URL":"https://docs.google.com/spreadsheets/d/1QsZqlJ_vZC5fYIKnSFdhkUxi-UAIRNzTB24DDyoZMhU/edit?gid=980819314#gid=980819314",
                    "QAFlag":""}


cfg = {
    "input": 
        {
            "required_tabs": ["ios","android"],
            "ios_header_rows": 3, 
            "android_header_rows": 3
        },
    ##add more formatting data for header rows
    "char_limit_policy": "strict",
    "output_sheets":
        ["formatted ios", "formatted android", "long results",'wide results'],
    ##add more formatting info for output sheets
    }"""

In [0]:
localizer = ASOLocalizer(request = request, gsheet_client =gsheet_client, gpt_client = gpt_client, cfg=cfg)

In [0]:
#localizer.run()